<a href="https://colab.research.google.com/github/NicoWeiner/MasterProject/blob/master/Funda_Text_Classification_%26_LIWC_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Funda - Text Classification & LIWC analysis

## 1.  Importing packages & the Funda scraped data 




In [0]:
# !pip install pandas 
# !pip install gensim
# !pip install spacy
# !pip install gensim
# !pip install matplotlib
# !pip install sklearn
# !pip install keras
# !pip install tensorflow
# !pip install scikit-learn
# !pip install scipy
# !pip install nltk
# !pip install random2
# !pip install warnings-plugin
# !pip install statsmodels
# !pip install seaborn
# !pip install DateTime
# !pip install datetime3

import numpy as np
import spacy
from spacy import displacy
import matplotlib.pyplot as plt
import keras
import tensorflow
import statsmodels.api as sm
import sklearn
import sklearn.preprocessing
from sklearn.linear_model import LinearRegression
from scipy import stats
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import random
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from pandas import DataFrame, read_csv
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import warnings
import os



Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
fundadf = pd.read_csv("https://www.dropbox.com/s/etowgsprsek92wu/FINAL_FUNDA.csv?dl=1", sep='\t', error_bad_lines=False, dtype='unicode', lineterminator='\n')
pd.options.display.max_columns = None
fundadf.head(5)


,globalid,publicationdate,postcode,postcodelong,sellingprice,description,housetype,categoryhouse,constructionyear,gardenindication,totalarea,livingarea,dateofselling,city,province,country,ID,Neighborhoods,Municipality,Men,Women,0-15 years,15-25 years,25-45 years,45-65 years,65 years or older,one persons household,Households without kids,Households with kids,Population density,Buy houses,Rent houses,Total theft from home and barn,Average_income,timetosell
0,4132354,2018-03-07,1011,1011VK,410000,OPEN AVOND 12 APRIL TUSSEN 17:00 - 19:00uur VR...,<{tussenverdieping}> (<{appartement}>),<{Appartement}>,1964,0,NaN,53,2018-08-08,Amsterdam,Noord Holland,Netherlands,860,GM0363,Amsterdam,423114,430933,126007,109354,304288,208818,105580,258090,96088,113428,5160,30,70,5,28700,154 days 00:00:00.000000000
1,4087960,2018-02-01,1108,1108HC,175000,Indien u de woning zou willen bekijken mailt u...,<{portiekflat}> (<{appartement}>),<{Appartement}>,1978,0,NaN,72,2018-02-19,Amsterdam Zuidoost,Noord Holland,Netherlands,860,GM0363,Amsterdam,423114,430933,126007,109354,304288,208818,105580,258090,96088,113428,5160,30,70,5,28700,18 days 00:00:00.000000000
2,4096821,2018-02-09,3315,3315MT,279500,"Op aantrekkelijke locatie, met uitzicht op een...",<{eengezinswoning}> <{hoekwoning}>,<{Woonhuis}>,1994,1,201,130,2018-04-30,Dordrecht,Zuid Holland,Netherlands,4041,GM0505,Dordrecht,58577,59849,18872,14053,29711,33761,22029,21557,14869,18338,1508,56,43,3,24600,80 days 00:00:00.000000000
3,4134064,2018-03-08,9732,9732AM,139500,MEERPAAL 208 TE GRONINGEN\n\nEen schitterend e...,<{portiekflat}>,<{Appartement}>,1972,0,NaN,98,2018-03-19,Groningen,Groningen,Netherlands,5925,GM0014,Groningen,101299,101511,24313,47673,61815,43312,25697,75483,25488,21570,2137,38,61,4,22400,11 days 00:00:00.000000000
4,4089019,2018-02-02,6433,6433DE,239000,In de jonge en kindvriendelijke wijk Overbroek...,<{eengezinswoning}> <{2-onder-1-kapwoning}>,<{Woonhuis}>,2010,1,230,108,2018-03-30,Hoensbroek,Limburg,Netherlands,6858,GM0917,Heerlen,43100,43662,11132,9093,20368,26737,19432,20591,12405,12034,1931,46,52,5,22500,56 days 00:00:00.000000000


## 2.  Run NFM Model to extract common topics in listing descriptions



### 2.1. Removing all Dutch stopwords in the listing descriptions as we only want to extract the relevant words

In [0]:
stopWords = set (stopwords.words('dutch'))

print(len(stopWords))
print(stopWords)

101
{'nu', 'ons', 'er', 'kan', 'andere', 'zijn', 'daar', 'nog', 'om', 'dus', 'iemand', 'ben', 'heeft', 'kon', 'je', 'der', 'uw', 'een', 'hier', 'over', 'dat', 'zelf', 'hij', 'meer', 'u', 'door', 'tegen', 'hebben', 'toch', 'voor', 'werd', 'met', 'worden', 'doen', 'zou', 'toen', 'was', 'ze', 'ook', 'geen', 'had', 'hoe', 'geweest', 'zij', 'ik', 'hem', 'en', 'altijd', 'alles', 'doch', 'waren', 'men', 'iets', 'maar', 'dit', 'mij', 'zo', 'de', 'is', 'onder', 'ja', 'omdat', 'wil', 'mijn', 'haar', 'na', 'uit', 'aan', 'heb', 'zonder', 'eens', 'wezen', 'als', 'zal', 'veel', 'het', 'hun', 'niet', 'deze', 'reeds', 'wie', 'tot', 'niets', 'of', 'in', 'dan', 'moet', 'kunnen', 'want', 'naar', 'al', 'zich', 'wat', 'die', 'te', 'op', 'ge', 'bij', 'wordt', 'me', 'van'}


In [0]:
tfidf_vect = TfidfVectorizer(max_df=0.8, min_df=2, stop_words=stopWords, token_pattern='[a-zA-Z][a-zA-Z]{2,}')
doc_term_matrix = tfidf_vect.fit_transform(fundadf['description'].values.astype('U'))


### 2.2. Creating a probability matrix for topics

In [0]:
nmf = NMF(n_components=25, random_state=42)
nmf.fit(doc_term_matrix)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=25, random_state=42, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [0]:
#retrieve the probability vector of words 

first_topic = nmf.components_[1]
top_topic_words = first_topic.argsort()[-10:]

### 2.3. Extracting top words for 25 topics

In [0]:
#the words in the data grouped by topics

for i,topic in enumerate(nmf.components_):
    print(f'Top words for topic #{i}:')
    print([tfidf_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top words for topic #0:
['achterzijde', 'stenen', 'overloop', 'slaapkamer', 'gelegen', 'voorzijde', 'berging', 'achterom', 'ruime', 'achtertuin']


Top words for topic #1:
['are', 'has', 'for', 'you', 'apartment', 'room', 'floor', 'with', 'and', 'the']


Top words for topic #2:
['centrale', 'per', 'maand', 'servicekosten', 'appartementencomplex', 'berging', 'lift', 'parkeerplaats', 'complex', 'appartement']


Top words for topic #3:
['waar', 'mooie', 'heerlijk', 'kunt', 'extra', 'ligt', 'ruimte', 'huis', 'grote', 'tuin']


Top words for topic #4:
['beperkingen', 'interpretatieverschillen', 'afrondingen', 'meetuitkomsten', 'eenduidige', 'meten', 'gebruiksoppervlakte', 'meting', 'nen', 'meetinstructie']


Top words for topic #5:
['collega', 'adressen', 'interesse', 'tijd', 'belang', 'geld', 'schakel', 'bespaart', 'aankoopmakelaar', 'nvm']


Top words for topic #6:
['kapwoning', 'achtertuin', 'bijkeuken', 'vrijstaande', 'auto', 'royale', 'inpandige', 'ruime', 'oprit', 'garage']


Top word

In [0]:
# Append new column with topics as numerical values
test = fundadf

topic_values = nmf.transform(doc_term_matrix)
test['Topic'] = topic_values.argmax(axis=1)
test.head(3)

,globalid,publicationdate,postcode,postcodelong,sellingprice,description,housetype,categoryhouse,constructionyear,gardenindication,totalarea,livingarea,dateofselling,city,province,country,ID,Neighborhoods,Municipality,Men,Women,0-15 years,15-25 years,25-45 years,45-65 years,65 years or older,one persons household,Households without kids,Households with kids,Population density,Buy houses,Rent houses,Total theft from home and barn,Average_income,timetosell,Topic
0,4132354,2018-03-07,1011,1011VK,410000,OPEN AVOND 12 APRIL TUSSEN 17:00 - 19:00uur VR...,<{tussenverdieping}> (<{appartement}>),<{Appartement}>,1964,0,NaN,53,2018-08-08,Amsterdam,Noord Holland,Netherlands,860,GM0363,Amsterdam,423114,430933,126007,109354,304288,208818,105580,258090,96088,113428,5160,30,70,5,28700,154 days 00:00:00.000000000,16
1,4087960,2018-02-01,1108,1108HC,175000,Indien u de woning zou willen bekijken mailt u...,<{portiekflat}> (<{appartement}>),<{Appartement}>,1978,0,NaN,72,2018-02-19,Amsterdam Zuidoost,Noord Holland,Netherlands,860,GM0363,Amsterdam,423114,430933,126007,109354,304288,208818,105580,258090,96088,113428,5160,30,70,5,28700,18 days 00:00:00.000000000,2
2,4096821,2018-02-09,3315,3315MT,279500,"Op aantrekkelijke locatie, met uitzicht op een...",<{eengezinswoning}> <{hoekwoning}>,<{Woonhuis}>,1994,1,201,130,2018-04-30,Dordrecht,Zuid Holland,Netherlands,4041,GM0505,Dordrecht,58577,59849,18872,14053,29711,33761,22029,21557,14869,18338,1508,56,43,3,24600,80 days 00:00:00.000000000,22


### 2.4. Give suitable topics a title 

Considering the output of the 25 topics above, we now select the ones that make sense and are suitable and categorize them with a name. The category name tells what a house description is focusing on. The ones that are do not make sense are later being dropped. 


In [0]:
# Insert the output in dataset as new variables

test.Topic = test.Topic.astype('str')

test.Topic = test.Topic.replace(['2'],'appartementencomplex')
test.Topic = test.Topic.replace(['3'],'trivialising')
test.Topic = test.Topic.replace(['4'],'measurements')
test.Topic = test.Topic.replace(['6', '13', '21'],'luxurious')
test.Topic = test.Topic.replace(['10'],'plotarea')
test.Topic = test.Topic.replace(['11'],'antique_charme')
test.Topic = test.Topic.replace(['16'],'central_location')
test.Topic = test.Topic.replace(['18'],'house_materials')
test.Topic = test.Topic.replace(['22', '24'],'great_view')

In [0]:
test = test.astype('str')


## 3. Import results of LIWC analysis & merge with Immoscout data

Outside of this notebook, the program LIWC2015 has been purchased which allowed to run multi-level linguistic analysis on the listing descriptions. The analysis has been conducted within the program LIWC2015, resulting in a new dataset with new columns that indicate the level of various linguistic features and sentiment. For more details on the particular linguistic feature, see: https://liwc.wpengine.com/compare-dictionaries/

In [0]:
liwc1 = pd.read_csv("https://www.dropbox.com/s/5ktrb9h7qwohsja/LIWC2015%20Results%202%28FINAL_FUNDA.csv%29.csv?dl=1", sep='\t', error_bad_lines=False, dtype='unicode', lineterminator='\n')
pd.options.display.max_columns = None

In [0]:
liwc1 = liwc1.drop(labels=0, axis=0)

In [0]:
liwc2 = liwc1
liwc2 = liwc2.drop(['B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'AA', 'AB', 'AC', 'AD', 'AE', 'AF', 'AG', 'AH', 'AI', 'filler\r'], axis=1) 
liwc2 = liwc2.astype('str')


In [0]:
liwc2_merged = pd.merge(test, liwc2, how="left", left_on="globalid", right_on="A")

In [0]:
liwc2_merged

,globalid,publicationdate,postcode,postcodelong,sellingprice,description,housetype,categoryhouse,constructionyear,gardenindication,totalarea,livingarea,dateofselling,city,province,country,ID,Neighborhoods,Municipality,Men,Women,0-15 years,15-25 years,25-45 years,45-65 years,65 years or older,one persons household,Households without kids,Households with kids,Population density,Buy houses,Rent houses,Total theft from home and barn,Average_income,timetosell,Topic,A,WC,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,prep,auxverb,adverb,conj,negate,verb,adj,compare,interrog,number,quant,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achieve,power,reward,risk,focuspast,focuspresent,focusfuture,relativ,motion,space,time,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu
0,4132354,2018-03-07,1011,1011VK,410000,OPEN AVOND 12 APRIL TUSSEN 17:00 - 19:00uur VR...,<{tussenverdieping}> (<{appartement}>),<{Appartement}>,1964,0,nan,53,2018-08-08,Amsterdam,Noord Holland,Netherlands,860,GM0363,Amsterdam,423114,430933,126007,109354,304288,208818,105580,258090,96088,113428,5160,30,70,5,28700,154 days 00:00:00.000000000,central_location,4132354,782,13.48,28.13,71.87,49.49,7.80,3.20,0.00,0.38,1.79,0.51,0.38,3.32,12.28,17.39,5.50,4.73,5.75,0.38,10.49,5.12,1.92,0.90,2.56,1.66,1.92,1.02,1.02,0.51,0.13,0.26,7.03,0.00,0.51,0.26,1.66,6.78,2.05,0.77,0.90,1.79,1.41,1.28,0.64,0.26,0.13,0.13,1.53,0.13,0.13,0.13,1.02,3.96,1.15,0.77,1.53,0.26,0.38,2.17,8.44,0.51,18.41,1.02,13.68,3.84,2.94,2.05,1.92,1.28,0.00,0.00,1.53,0.00,0.38,0.00,0.00
1,4087960,2018-02-01,1108,1108HC,175000,Indien u de woning zou willen bekijken mailt u...,<{portiekflat}> (<{appartement}>),<{Appartement}>,1978,0,nan,72,2018-02-19,Amsterdam Zuidoost,Noord Holland,Netherlands,860,GM0363,Amsterdam,423114,430933,126007,109354,304288,208818,105580,258090,96088,113428,5160,30,70,5,28700,18 days 00:00:00.000000000,appartementencomplex,4087960,448,17.23,30.13,67.86,45.98,3.57,2.01,0.00,0.67,1.34,0.00,0.00,0.89,13.84,16.29,5.36,2.01,4.91,0.00,9.82,4.69,2.23,0.22,5.58,1.79,1.34,0.00,1.34,1.12,0.00,0.22,4.91,0.00,0.45,0.00,0.67,4.91,1.79,0.22,0.89,0.89,0.89,0.67,1.12,0.89,0.00,0.00,0.45,0.00,0.00,0.00,0.22,2.23,1.34,0.22,0.22,0.22,0.22,2.90,7.81,1.56,18.53,0.89,13.84,4.46,0.89,0.67,4.24,0.00,0.00,0.00,0.67,0.00,0.00,0.00,0.00
2,4096821,2018-02-09,3315,3315MT,279500,"Op aantrekkelijke locatie, met uitzicht op een...",<{eengezinswoning}> <{hoekwoning}>,<{Woonhuis}>,1994,1,201,130,2018-04-30,Dordrecht,Zuid Holland,Netherlands,4041,GM0505,Dordrecht,58577,59849,18872,14053,29711,33761,22029,21557,14869,18338,1508,56,43,3,24600,80 days 00:00:00.000000000,great_view,4096821,579,13.47,29.53,67.70,44.21,3.97,2.59,0.00,0.35,0.52,0.17,0.17,2.76,11.74,16.06,4.32,3.80,6.56,0.35,7.94,4.84,2.25,1.21,6.91,0.86,3.45,1.73,1.73,1.38,0.00,0.17,2.25,0.00,0.00,0.00,0.52,6.74,1.38,0.52,1.38,1.90,0.86,1.90,1.38,0.86,0.00,0.52,1.04,0.00,0.17,0.00,0.86,2.42,0.69,0.17,0.35,0.86,0.35,1.73,6.39,0.86,14.68,0.69,10.02,3.97,0.86,3.63,5.53,0.17,0.00,0.00,0.52,0.00,0.17,0.00,0.00
3,4134064,2018-03-08,9732,9732AM,139500,MEERPAAL 208 TE GRONINGEN\n\nEen schitterend e...,<{portiekflat}>,<{Appartement}>,1972,0,nan,98,2018-03-19,Groningen,Groningen,Netherlands,5925,GM0014,Groningen,101299,101511,24313,47673,61815,43312,25697,75483,25488,21570,2137,38,61,4,22400,11 days 00:00:00.000000000,14,4134064,528,16.50,27.27,69.51,50.38,6.25,2.46,0.00,0.76,0.95,0.00,0.00,3.60,13.07,16.48,6.06,4.17,6.44,1.52,9.47,6.63,3.79,0.76,3.60,1.70,2.65,1.70,0.95,0.57,0.00,0.00,5.49,0.00,0.00,0.00,1.70,9.47,1.70,0.76,1.14,2.46,1.33,3.41,1.14,0.76,0.19,0.19,1.52,0.00,0.19,0.00,0.95,3.22,0.76,0.95,0.76,0.38,0.57,1.70,7.77,0.95,12.50,0.38,8.52,3.79,1.89,0.76,3.98,1.14,0.00,0.00,1.14,0.00,0.19,0.00,0.00
4,4089019,2018-02-02,6433,6433DE,239000,In de jonge en kindvriendelijke wijk

In [0]:
# liwc2_merged2 = liwc2_merged.drop(['A', 'publicationdate', 'postcode', 'postcodelong', 'description', 'housetype', 'dateofselling', 'country', 'ID', 'province', 'Neighborhoods', 'Municipality'], axis=1) 


liwc2_merged2 = liwc2_merged.drop(['A', 'postcode', 'housetype', 'country', 'ID', 'Neighborhoods', 'Municipality'], axis=1) 








Now we create a column that categorizes sellingprice into 5 percentiles (0.2 = very low, 0.4 = low, 0.6 = medium, 0.8 = high, 1 = very high) to get better results in the regression and neural network.

In [0]:
liwc2_merged2.sellingprice = pd.to_numeric(liwc2_merged.sellingprice, errors='coerce').fillna(0).astype(np.int64)


In [0]:
liwc2_merged2['pricerange'] = pd.qcut(liwc2_merged2['sellingprice'], 5, labels=["0.2", "0.4", "0.6", "0.8", "1.0"])
col = "pricerange" 
liwc2_merged2 = pd.concat([liwc2_merged2[col],liwc2_merged2.drop(col,axis=1)], axis=1)

In [0]:
liwc3_merged = liwc2_merged2.set_index('globalid')

In [0]:
liwc4 = liwc3_merged
liwc4['constructionyear'] = liwc4['constructionyear'].str.replace('-', '') .str.replace('<{Voor}>', '') .str.replace("<{Na}>", '')
liwc4['constructionyear'] = liwc4['constructionyear'].str[:4]
liwc4['categoryhouse'] = liwc4["categoryhouse"].str.replace("<{", "") .str.replace("}>", "")
liwc4['Average_income'] = liwc4["Average_income"].str.replace(" ", "NaN")
liwc4['timetosell'] = liwc4['timetosell'].str[:4]
liwc4['timetosell'] = liwc4['timetosell'].str.replace(r'\D', '')

In [0]:
liwc4

,pricerange,publicationdate,postcodelong,sellingprice,description,categoryhouse,constructionyear,gardenindication,totalarea,livingarea,dateofselling,city,province,Men,Women,0-15 years,15-25 years,25-45 years,45-65 years,65 years or older,one persons household,Households without kids,Households with kids,Population density,Buy houses,Rent houses,Total theft from home and barn,Average_income,timetosell,Topic,WC,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,prep,auxverb,adverb,conj,negate,verb,adj,compare,interrog,number,quant,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achieve,power,reward,risk,focuspast,focuspresent,focusfuture,relativ,motion,space,time,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu
globalid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4132354,1.0,2018-03-07,1011VK,410000,OPEN AVOND 12 APRIL TUSSEN 17:00 - 19:00uur VR...,Appartement,1964,0,nan,53,2018-08-08,Amsterdam,Noord Holland,423114,430933,126007,109354,304288,208818,105580,258090,96088,113428,5160,30,70,5,28700,154,central_location,782,13.48,28.13,71.87,49.49,7.80,3.20,0.00,0.38,1.79,0.51,0.38,3.32,12.28,17.39,5.50,4.73,5.75,0.38,10.49,5.12,1.92,0.90,2.56,1.66,1.92,1.02,1.02,0.51,0.13,0.26,7.03,0.00,0.51,0.26,1.66,6.78,2.05,0.77,0.90,1.79,1.41,1.28,0.64,0.26,0.13,0.13,1.53,0.13,0.13,0.13,1.02,3.96,1.15,0.77,1.53,0.26,0.38,2.17,8.44,0.51,18.41,1.02,13.68,3.84,2.94,2.05,1.92,1.28,0.00,0.00,1.53,0.00,0.38,0.00,0.00
4087960,0.2,2018-02-01,1108HC,175000,Indien u de woning zou willen bekijken mailt u...,Appartement,1978,0,nan,72,2018-02-19,Amsterdam Zuidoost,Noord Holland,423114,430933,126007,109354,304288,208818,105580,258090,96088,113428,5160,30,70,5,28700,18,appartementencomplex,448,17.23,30.13,67.86,45.98,3.57,2.01,0.00,0.67,1.34,0.00,0.00,0.89,13.84,16.29,5.36,2.01,4.91,0.00,9.82,4.69,2.23,0.22,5.58,1.79,1.34,0.00,1.34,1.12,0.00,0.22,4.91,0.00,0.45,0.00,0.67,4.91,1.79,0.22,0.89,0.89,0.89,0.67,1.12,0.89,0.00,0.00,0.45,0.00,0.00,0.00,0.22,2.23,1.34,0.22,0.22,0.22,0.22,2.90,7.81,1.56,18.53,0.89,13.84,4.46,0.89,0.67,4.24,0.00,0.00,0.00,0.67,0.00,0.00,0.00,0.00
4096821,0.6,2018-02-09,3315MT,279500,"Op aantrekkelijke locatie, met uitzicht op een...",Woonhuis,1994,1,201,130,2018-04-30,Dordrecht,Zuid Holland,58577,59849,18872,14053,29711,33761,22029,21557,14869,18338,1508,56,43,3,24600,80,great_view,579,13.47,29.53,67.70,44.21,3.97,2.59,0.00,0.35,0.52,0.17,0.17,2.76,11.74,16.06,4.32,3.80,6.56,0.35,7.94,4.84,2.25,1.21,6.91,0.86,3.45,1.73,1.73,1.38,0.00,0.17,2.25,0.00,0.00,0.00,0.52,6.74,1.38,0.52,1.38,1.90,0.86,1.90,1.38,0.86,0.00,0.52,1.04,0.00,0.17,0.00,0.86,2.42,0.69,0.17,0.35,0.86,0.35,1.73,6.39,0.86,14.68,0.69,10.02,3.97,0.86,3.63,5.53,0.17,0.00,0.00,0.52,0.00,0.17,0.00,0.00
4134064,0.2,2018-03-08,9732AM,139500,MEERPAAL 208 TE GRONINGEN\n\nEen schitterend e...,Appartement,1972,0,nan,98,2018-03-19,Groningen,Groningen,101299,101511,24313,47673,61815,43312,25697,75483,25488,21570,2137,38,61,4,22400,11,14,528,16.50,27.27,69.51,50.38,6.25,2.46,0.00,0.76,0.95,0.00,0.00,3.60,13.07,16.48,6.06,4.17,6.44,1.52,9.47,6.63,3.79,0.76,3.60,1.70,2.65,1.70,0.95,0.57,0.00,0.00,5.49,0.00,0.00,0.00,1.70,9.47,1.70,0.76,1.14,2.46,1.33,3.41,1.14,0.76,0.19,0.19,1.52,0.00,0.19,0.00,0.95,3.22,0.76,0.95,0.76,0.38,0.57,1.70,7.77,0.95,12.50,0.38,8.52,3.79,1.89,0.76,3.98,1.14,0.00,0.00,1.14,0.00,0.19,0.00,0.00
4089019,0.4,2018-02-02,6433DE,239000,In de jonge en kindvriendelijke wijk Overbroek...,Woonhuis,2010,1,230,108,2018-03-30,Hoensbroek,Limburg,43100,43662,11132,9093,20368,26737,19432,20591,12405,12034,1931,46,52,5,22500,56,14,421,70.17,35.15,51.78,31.12,1.66,0.24,0.00,0.00,0.00,0.24,0.24,0.71,5.23,13.78,3.09,3.80,5.23,1.19,8.79,2.38,1.66,0.24,13.30,1.43,0.71,0.48,0.24,0.24,0.00,0.00,5.23,0.00,0.00,0.00,1.19,5.23,0.71,0.71,0.71,0.

In [0]:
liwc4.to_excel('LIWC_Funda_complete.xlsx')

In [0]:
liwc4.to_csv('LIWC_Funda_complete.csv')